In [72]:
import urllib.request
import urllib.error
import json
import time
import re
import os
import csv
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome('C:/Users/yanli/Desktop/coding/python/spider/chromedriver')
url = "https://search.jd.com/search?keyword=%E4%B8%89%E6%98%9F&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=%E4%B8%89%E6%98%9F&ev=exbrand_%E4%B8%89%E6%98%9F%EF%BC%88SAMSUNG%EF%BC%89%5E&cid3=655#J_searchWrap"
driver.get(url)
# actions=ActionChains(driver)
main_links = []
page_number = 1
itemCnt = 1

In [73]:
actions=ActionChains(driver)

In [74]:
while True:
    for _ in range(3):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)
    # Find all hotels in the container
    for element in driver.find_elements_by_class_name("gl-item"):            
        links = element.find_element_by_tag_name("a")
        href = links.get_attribute('href')
        main_links.append(href) 
        itemCnt += 1
    try:
        link = driver.find_element_by_xpath('//*[@id="J_bottomPage"]/span[1]/a[9]')
    except NoSuchElementException:
        break     
    link.click()
    driver.get(driver.current_url)
    # While True, scrap next page
    page_number += 1
    time.sleep(2)

KeyboardInterrupt: 

In [71]:
prev_url=url
while True:
    for _ in range(3):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)
    # Find all hotels in the container
    for element in driver.find_elements_by_class_name("gl-item"):            
        links = element.find_element_by_tag_name("a")
        href = links.get_attribute('href')
        main_links.append(href) 
        itemCnt += 1
    try:
        link = driver.find_element_by_xpath('//*[@id="J_bottomPage"]/span[1]/a[9]')
    except NoSuchElementException:
        break     
    link.click()
    driver.get(driver.current_url)
    if prev_url==driver.get(driver.current_url):break
    else:# While True, scrap next page
        page_number += 1
        prev_url=driver.get(driver.current.url)
        time.sleep(2)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.16299 x86_64)


In [75]:
len(main_links)

950

In [13]:
from collections import OrderedDict
main_links = list(dict.fromkeys(main_links))

In [56]:
import requests
import urllib3

requests.packages.urllib3.disable_warnings()
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

page = requests.get(url, verify=False)

In [16]:
def url_open(url):
    try: 
        urllib.request.urlopen(url)
    except urllib.error.HTTPError as e:
        print("HTTP ERROR")
        print("error code:",e.code)
    except urllib.error.URLError as e:
        print("CAN'T REACH A SERVER")
        print("error:",e.reason)
    else:
        req=urllib.request.Request(url)
        req.add_header('User-Agent',"Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1")
        response=urllib.request.urlopen(req)
        html=response.read()
        return html

In [39]:
def url_open(url):
    try: 
        urllib.request.urlopen(url)
    except urllib.error.HTTPError as e:
        print("HTTP ERROR")
        print("error code:",e.code)
    except urllib.error.URLError as e:
        print("CAN'T REACH A SERVER")
        print("error:",e.reason)
    else:
        req=urllib.request.Request(url)
        req.add_header('User-Agent',"Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1")
        response=urllib.request.urlopen(req)
        html=response.read()
        response.close()  
        return html

In [18]:
def get_item(url):
    pattern=r'target="_blank" href="(//item.jd.com/[^"]+\.html)'
    items_url=re.findall(pattern,url)
    return items_url

In [19]:
def get_long_name(item_html):
    #item_html=url_open("https:"+item_url).decode("gbk")
    item_name=re.findall(r'当季新品|京东物流|京东精选',item_html)
    if len(item_name)!= 0:
        pattern=r'<div class="sku-name">\n[^<]+<img src="//[^>]+>\n([^"]+)</div>\n                        <div cl'
    else:
        pattern=r'<div class="sku-name">\n([^"]+)</div>'
    item_name=re.findall(pattern,item_html)
    return item_name

In [20]:
def get_name(item_html):
    pattern=r'商品名称：([^<]+)<'
    item_name=re.findall(pattern,item_html)
    return item_name

In [85]:
item_html=url_open("https://item.jd.hk/1984148414.html").decode("gbk")
print(item_html)
pattern=r'<ul class="parameter2">'
print("name",re.findall(pattern,item_html))

<input type="hidden" id="returnurl" value="http://mitem.jd.hk/ware/view.action?wareId=1984148414"/>
<script>
            location.href="/unifiedlogin/checkcredentials.action?returnUrl=http://mitem.jd.hk/ware/view.action?wareId=1984148414";
     </script>


name []


In [ ]:
print("RAM",get_storage(item_html))
print("battery",get_battery_capacity(item_html))
print("color",get_color(item_html))
front,rear=get_camera_piexl(item_html)
print("front",front)
print("rear",rear)

In [21]:
def get_price(item_url):
    item_id=re.findall(r'//item.jd.com/([^.]+)\.html',item_url)
    url="http://p.3.cn/prices/mgets?skuIds=J_"+str(item_id[0])
    price_html=url_open(url).decode("gbk")
    item_price=re.findall(r'"p":"([^"]+)"',price_html)
    return item_price

In [22]:
def get_storage(item_html):
    #item_html=url_open("https:"+item_url).decode("gbk")
    pattern=r'([^>]+)<[^<]+<dt>存储卡</dt>'
    item_storage=re.findall(pattern,item_html)
    return item_storage

In [23]:
def get_battery_capacity(item_html):
    #item_html=url_open("https:"+item_url).decode("gbk")
    pattern=r'<dt>电池容量（mAh）</dt><dd>([^<]+)<'
    battery_capacity=re.findall(pattern,item_html)
    return battery_capacity

In [25]:
def get_color(item_html):
    #item_html=url_open("https:"+item_url).decode("gbk")
    pattern=r'机身颜色</dt><dd>([^<]+)<'
    item_color=re.findall(pattern,item_html)
    return item_color

In [27]:
def get_camera_piexl(item_html):
    #item_html=url_open("https:"+item_url).decode("gbk")
    pattern_front=r'<dt>前置摄像头</dt><dd>([^<]+)<'
    pattern_rear=r'<dt>后置摄像头</dt><dd>([^<]+)<'
    front_pixel=re.findall(pattern_front,item_html)
    rear_pixel=re.findall(pattern_rear,item_html)
    return front_pixel,rear_pixel

In [28]:
def get_next_page_url(current_url):
    page=re.findall(r'page=(\d)',current_url)[0]
    page=str(int(page)+2)
    next_url="https://search.jd.com/search?keyword=samsung&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&bs=1&wq=samsung&cid2=653&cid3=655&ev=exbrand_%E4%B8%89%E6%98%9F%EF%BC%88SAMSUNG%EF%BC%89%5E&page="+page+"&s=58&click=0"
    return next_url

In [29]:
def read_current_url(current_url):
    html=url_open(current_url).decode("utf-8")
    item_url=get_item(html)
    return item_url

In [42]:
name=[]
price=[]
RAM=[]
battery=[]
color=[]
front_pixel=[]
rear_pixel=[]
count=1
if __name__=='__main__':
    for each in main_links:
        print(count)
        if(len(each)<50):
            item_html=url_open(each).decode("gb18030")
            name.append(get_name(item_html))
            price.append(get_price(each))
            RAM.append(get_storage(item_html))
            battery.append(get_battery_capacity(item_html))
            color.append(get_color(item_html))
            front,rear=get_camera_piexl(item_html)
            front_pixel.append(front)
            rear_pixel.append(rear)
        time.sleep(2)
        count+=1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


UnicodeDecodeError: 'gb18030' codec can't decode byte 0x89 in position 151: illegal multibyte sequence

In [86]:
if os.path.exists("data.csv"):
    os.remove("data.csv")
csvFile = open("data.csv", "w")
writer = csv.writer(csvFile)
writer.writerow(name)
writer.writerow(price)
writer.writerow(RAM)
writer.writerow(battery)
writer.writerow(color)
writer.writerow(front_pixel)
writer.writerow(rear_pixel)
csvFile.close()